In [34]:
import re
import tarfile
import functools

import numpy as np
import spacy

from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

In [35]:
nlp = spacy.load('en')

In [36]:
def tokenize(sent):
    word_tokens = nlp(sent)
    word_list = []
    POS_list = []
    for words in word_tokens:
        POS_list.append(words.pos)
        word_list.append(words.text)
    return word_list, POS_list

In [37]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true,
    only the sentences that support the answer are kept.
    '''
    data = []
    story = []
    storyPOS = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
            storyPOS = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q, qPos = tokenize(q)
            substory = None
            substoryPOS = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
                substoryPOS = [x for x in storyPOS if x]
            data.append((substory, substoryPOS, q, qPos, a))
            story.append('')
        else:
            sent, sentPOS = tokenize(line)
            story.append(sent)
            storyPOS.append(sentPOS)
    return data

In [38]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file, retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: functools.reduce(lambda x, y: x + y, data)
    data = [(flatten(story), flatten(storyPOS), q, qPos, answer) 
            for story, storyPOS, q, qPos, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [39]:
def vectorize_stories(data, word_idx, wordPOS_idx, story_maxlen, query_maxlen):
    xs = []
    xspos = []
    xqs = []
    xqspos = []
    ys = []
    for story, storyPOS, query, queryPOS, answer in data:
        x = [word_idx[w] for w in story]
        xPOS = [wordPOS_idx[w] for w in storyPOS]
        xq = [word_idx[w] for w in query]
        xqPOS = [wordPOS_idx[w] for w in queryPOS]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        xs.append(x)
        xspos.append(xPOS)
        xqs.append(xq)
        xqspos.append(xqPOS)
        ys.append(y)
    return pad_sequences(xs, maxlen=story_maxlen), pad_sequences(xspos, maxlen=story_maxlen), pad_sequences(xqs, maxlen=query_maxlen), pad_sequences(xqspos, maxlen=query_maxlen), np.array(ys)

In [58]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
SENT_HIDDEN_SIZE = 100
QUERY_HIDDEN_SIZE = 100
BATCH_SIZE = 32
EPOCHS = 20
print('RNN / Embed / Sent / Query = {}, {}, {}, {}'.format(RNN,
                                                           EMBED_HIDDEN_SIZE,
                                                           SENT_HIDDEN_SIZE,
                                                           QUERY_HIDDEN_SIZE))

RNN / Embed / Sent / Query = <class 'keras.layers.recurrent.LSTM'>, 50, 100, 100


In [59]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [60]:
# Default QA1 with 1000 samples
challenge = 'tasks_1-20_v1-2/en/qa1_single-supporting-fact_{}.txt'
# QA1 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt'
# QA2 with 1000 samples
#challenge = 'tasks_1-20_v1-2/en/qa2_two-supporting-facts_{}.txt'
# QA2 with 10,000 samples
# challenge = 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt'
train = get_stories(tar.extractfile(challenge.format('train')))
test = get_stories(tar.extractfile(challenge.format('test')))

In [61]:
vocab = set()
vocabPOS = set()
for story, storyPOS, q, qPos, answer in train + test:
    vocab |= set(story + q + [answer])
    vocabPOS |= set(storyPOS + qPos)
vocab = sorted(vocab)

In [62]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
vocabPOS_size = len(vocabPOS) + 1
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
wordPOS_idx = dict((c, i + 1) for i, c in enumerate(vocabPOS))
story_maxlen = max(map(len, (x for x, _, _, _, _ in train + test)))
query_maxlen = max(map(len, (x for _, _, x, _, _ in train + test)))

In [63]:
x, xpos, xq, xqpos, y = vectorize_stories(train, word_idx, wordPOS_idx, story_maxlen, query_maxlen)
tx, txpos, txq, txqpos, ty = vectorize_stories(test, word_idx, wordPOS_idx, story_maxlen, query_maxlen)

In [64]:
print('vocab = {}'.format(vocab))
print('vocabPOS = {}'.format(vocabPOS))
print('x.shape = {}'.format(x.shape))
print('xpos.shape = {}'.format(xpos.shape))
print('xq.shape = {}'.format(xq.shape))
print('xqpos.shape = {}'.format(xqpos.shape))
print('y.shape = {}'.format(y.shape))
print('story_maxlen, query_maxlen = {}, {}'.format(story_maxlen, query_maxlen))

vocab = ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']
vocabPOS = {98, 83, 84, 88, 90, 94, 95}
x.shape = (1000, 66)
xpos.shape = (1000, 66)
xq.shape = (1000, 4)
xqpos.shape = (1000, 4)
y.shape = (1000, 22)
story_maxlen, query_maxlen = 66, 4


In [65]:
print('Build model...')

# Input the story as a sequence of tokens
sentence = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_sentence = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(sentence)
sentence_pos = layers.Input(shape=(story_maxlen,), dtype='int32')
encoded_sentence_pos = layers.Embedding(vocabPOS_size, EMBED_HIDDEN_SIZE)(sentence_pos)

merged_sentence = layers.concatenate([encoded_sentence, encoded_sentence_pos])
merged_sentence = layers.Dropout(0.3)(merged_sentence)
merged_sentence = RNN(EMBED_HIDDEN_SIZE)(merged_sentence)

# Input the questions as a sequence of tokens
question = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question = layers.Embedding(vocab_size, EMBED_HIDDEN_SIZE)(question)
question_pos = layers.Input(shape=(query_maxlen,), dtype='int32')
encoded_question_pos = layers.Embedding(vocabPOS_size, EMBED_HIDDEN_SIZE)(question_pos)

merged_question = layers.concatenate([encoded_question, encoded_question_pos])
merged_question = layers.Dropout(0.3)(merged_question)
merged_question = RNN(EMBED_HIDDEN_SIZE)(merged_question)
#merged_question = layers.RepeatVector(story_maxlen)(merged_question)


merged = layers.concatenate([merged_sentence, merged_question]) 
merged = layers.Dropout(0.3)(merged)
#merged = RNN(EMBED_HIDDEN_SIZE)(merged)
preds = layers.Dense(vocab_size, activation='softmax')(merged) 

model = Model([sentence, sentence_pos, question, question_pos], preds)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Build model...


In [66]:
print('Training')
model.fit([x, xpos, xq, xqpos], y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05)
loss, acc = model.evaluate([tx, txpos, txq, txqpos], ty,
                           batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 950 samples, validate on 50 samples
Epoch 1/20
950/950 [==============================] - 8s - loss: 2.6757 - acc: 0.1453 - val_loss: 1.9266 - val_acc: 0.1800